In [84]:
import pandas as pd
import numpy as np
import pymysql
from sqlalchemy import create_engine
import getpass  # To get the password without showing the input
from sqlalchemy import text
password = getpass.getpass()

 ········


1) establish connection to sakila DB

In [86]:
db = "sakila"
connection_string = 'mysql+pymysql://root:' + password + '@localhost/' + db
engine = create_engine(connection_string)

with engine.connect() as connection:
    query = text('SELECT * FROM film')
    result = connection.execute(query)

print(result)

2)
Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
month: an integer representing the month for which rental data is to be retrieved.
year: an integer representing the year for which rental data is to be retrieved.
The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [88]:
def rentals_month(engine, month, year):
    with engine.connect() as connection:
        '''
        Retrieves rental data for a given month and year from the Sakila DB as a Pandas DataFrame.
        '''
        txt = f'''
            SELECT title, rental_date, rental_id, inventory_id, customer_id, return_date, staff_id
            FROM rental
            JOIN inventory
            USING(inventory_id)
            JOIN film
            USING (film_id)
            WHERE YEAR(rental_date) = {year} AND MONTH(rental_date) = {month};
        '''
        query = text(txt)
        result = connection.execute(query)
        return pd.DataFrame(result.fetchall(), columns=result.keys())

rentals_month_df = rentals_month(engine, 5, 2005)
rentals_month_df

,title,rental_date,rental_id,inventory_id,customer_id,return_date,staff_id
0,ACADEMY DINOSAUR,2005-05-30 20:21:07,972,2,411,2005-06-06 00:36:07,1
1,ACADEMY DINOSAUR,2005-05-27 07:03:28,361,6,587,2005-05-31 08:01:28,1
2,ADAPTATION HOLES,2005-05-31 04:50:07,1033,14,25,2005-06-02 01:53:07,1
3,AFFAIR PREJUDICE,2005-05-31 00:06:02,995,17,150,2005-06-06 02:30:02,2
4,AFFAIR PREJUDICE,2005-05-27 20:44:36,465,20,261,2005-06-02 02:43:36,1
...,...,...,...,...,...,...,...
1151,ZHIVAGO CORE,2005-05-30 05:15:20,873,4568,375,2005-06-07 00:49:20,2
1152,ZOOLANDER FICTION,2005-05-30 05:01:30,871,4573,303,2005-06-04 06:22:30,2
1153,ZORRO ARK,2005-05-31 11:10:17,1084,4577,12,2005-06-01 11:15:17,1
1154,ZORRO ARK,2005-05-28 11:33:46,582,4579,198,2005-05-29 08:33:46,1


3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

Hint: Consider making use of pandas groupby()

In [90]:
def rental_count_month(df, month, year):
    df['rental_date'] = df['rental_date'].astype(str)
    df_filtered = df[df['rental_date'].str.startswith(f'{year}-{month:02}')]
    df_new = df_filtered.groupby('customer_id').size().reset_index(name=f'rentals_{month:02}_{year}')
    return df_new
df = rental_count_month(rentals_month_df, 5, 2005)
df

,customer_id,rentals_05_2005
0,1,2
1,2,1
2,3,2
3,5,3
4,6,3
...,...,...
515,594,4
516,595,1
517,596,6
518,597,2


Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [102]:
rentals_month_df_may = rentals_month(engine, 5, 2005)
rentals_month_df_june = rentals_month(engine, 6, 2005)
rental_count_may = rental_count_month(rentals_month_df_may, 5, 2005)
rental_count_june = rental_count_month(rentals_month_df_june, 6, 2005)

In [108]:
def compare_rentals(df1, df2):
    df = pd.merge(df1, df2, on='customer_id', how='outer').fillna(0)
    rental_cols = [col for col in df.columns if col.startswith('rentals_')]
    df['difference'] = df[rental_cols[0]] - df[rental_cols[1]]
    #df['difference'] = abs(df[rental_cols[0]] - df[rental_cols[1]])
    return df
df = compare_rentals(rental_count_may, rental_count_june)
df

,customer_id,rentals_05_2005,rentals_06_2005,difference
0,1,2.0,7.0,5.0
1,2,1.0,1.0,0.0
2,3,2.0,4.0,2.0
3,5,3.0,5.0,2.0
4,6,3.0,4.0,1.0
...,...,...,...,...
593,583,0.0,6.0,6.0
594,585,0.0,4.0,4.0
595,591,0.0,3.0,3.0
596,592,0.0,5.0,5.0
